In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
from mlrl.experiments.train_maze_agent import (
    create_maze_meta_env, RestrictedActionsMazeState, create_batched_tf_meta_env, 
    create_agent, create_training_run, get_maze_name
)
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from tf_agents.environments.gym_wrapper import GymWrapper
from tf_agents.environments.batched_py_environment import BatchedPyEnvironment
from tf_agents.train.utils import spec_utils

args = {
    'agent': 'ppo_agent', 
    'meta_time_limit': 500,
}
env_batch_size = 2

env = BatchedPyEnvironment([
    GymWrapper(create_maze_meta_env(RestrictedActionsMazeState, args)) 
    for _ in range(env_batch_size)
], multithreading=False)

eval_env = BatchedPyEnvironment([
    GymWrapper(create_maze_meta_env(RestrictedActionsMazeState, args)) 
    for _ in range(env_batch_size)
], multithreading=False)

env.reset()

observation_tensor_spec, action_tensor_spec, time_step_tensor_spec = (
      spec_utils.get_tensor_specs(env))

pygame 2.1.0 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: N

In [3]:
from tf_agents.agents.ppo.ppo_agent import PPOAgent
from tf_agents.train.utils import train_utils
from tf_agents.networks.mask_splitter_network import MaskSplitterNetwork

from mlrl.meta.meta_env import mask_token_splitter
from mlrl.meta.search_networks import create_action_distribution_network
from mlrl.meta.search_networks import create_value_network

train_step = train_utils.create_train_step()

network_kwargs = {
    'n_heads': 3,
    'n_layers': 2,
    'head_dim': 32,
}

value_net=create_value_network(observation_tensor_spec, **network_kwargs)

actor_net = create_action_distribution_network(observation_tensor_spec['search_tree_tokens'],
                                               action_tensor_spec,
                                               **network_kwargs)

masked_actor_net = MaskSplitterNetwork(mask_token_splitter,
                                       actor_net,
                                       input_tensor_spec=observation_tensor_spec,
                                       passthrough_mask=True)


agent = PPOAgent(
    time_step_tensor_spec,
    action_tensor_spec,
    actor_net=masked_actor_net,
    value_net=value_net,
    optimizer=tf.keras.optimizers.Adam(3e-4),
    train_step_counter=train_step,
    compute_value_and_advantage_in_train=False,
    update_normalizers_in_train=False,
    normalize_observations=False,
    discount_factor=0.99,
    num_epochs=1,  # deprecated param
)

2022-10-04 23:05:10.498104: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-04 23:05:10.548942: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-04 23:05:10.551386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-04 23:05:10.552682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [4]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size or 1,
    max_length=10000
)


def preprocess_seq(experience, info):
    return agent.preprocess_sequence(experience), info


def dataset_fn():
    ds = replay_buffer.as_dataset(sample_batch_size=4, num_steps=64)
    return ds.map(preprocess_seq).prefetch(5)

In [5]:
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.metrics import py_metrics
from tf_agents.train import triggers

import os

root_dir = './runs/ppo_agent/test'

summary_interval = 1000
collect_sequence_length = 2048
policy_save_interval = 5000


saved_model_dir = os.path.join(root_dir, learner.POLICY_SAVED_MODEL_DIR)
collect_env_step_metric = py_metrics.EnvironmentSteps()
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        agent,
        train_step,
        interval=policy_save_interval,
        metadata_metrics={
            triggers.ENV_STEP_METADATA_KEY: collect_env_step_metric
        }),
    triggers.StepPerSecondLogTrigger(train_step, interval=summary_interval),
]

collect_actor = actor.Actor(
    env,
    agent.collect_policy,
    train_step,
    steps_per_run=collect_sequence_length,
    observers=[replay_buffer.add_batch],
    metrics=actor.collect_metrics(buffer_size=collect_sequence_length),
    reference_metrics=[collect_env_step_metric],
    summary_dir=os.path.join(root_dir, learner.TRAIN_DIR),
    summary_interval=summary_interval)

collect_actor.run()

In [6]:
from tf_agents.policies import py_tf_eager_policy

eval_steps=1000

eval_greedy_policy = py_tf_eager_policy.PyTFEagerPolicy(
    agent.policy, use_tf_function=True)

eval_actor = actor.Actor(
    eval_env,
    eval_greedy_policy,
    train_step,
    metrics=actor.eval_metrics(buffer_size=10),
    reference_metrics=[collect_env_step_metric],
    summary_dir=os.path.join(root_dir, 'eval'),
    steps_per_run=eval_steps)

# eval_actor.run_and_log()

In [7]:
from tf_agents.train.ppo_learner import PPOLearner

ppo_learner = PPOLearner(
    root_dir,
    train_step,
    agent,
    experience_dataset_fn=dataset_fn,
    normalization_dataset_fn=dataset_fn,
    num_samples=1, num_epochs=20,  # num samples * num epochs = num iterations per run call
    triggers=learning_triggers,
    shuffle_buffer_size=collect_sequence_length
)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [8]:
import wandb
from mlrl.utils.render_utils import create_policy_eval_video


videos_dir = root_dir + '/videos'

from pathlib import Path
Path(videos_dir).mkdir(parents=True, exist_ok=True) 


eval_interval = 5
num_iterations = 100

try:
      config = {
            'name': '5x5_maze_ppo',
            'eval_interval': eval_interval,
            'num_iterations': num_iterations,
            'summary_interval': summary_interval,
            'collect_sequence_length': collect_sequence_length,
            'policy_save_interval': policy_save_interval,
            'num_samples': ppo_learner._num_samples,
            'num_epochs': ppo_learner._num_epochs,
            'meta_discount_factor': agent._discount_factor,
            'max_tree_size': env.envs[0].max_tree_size,
            'env_batch_size': env.batch_size,
      }

      wandb.init(project='mlrl', entity='drcope', reinit=True, config=config)

      for i in range(num_iterations):
            iteration_logs = {'iteration': i}

            print(f'Iteration: {i}')

            # its very important to reset the actors
            # otherwise the observations can be wrong on the next run
            collect_actor.reset()
            eval_actor.reset()
            for metric in eval_actor.metrics:
                  metric.reset()
            for metric in collect_actor.metrics:
                  metric.reset()

            if i % eval_interval == 0:
                  eval_actor.run_and_log()
                  print('Evaluation stats:')
                  print(', '.join([f'{metric.name}: {metric.result():.3f}' for metric in eval_actor.metrics]))
                  iteration_logs.update({f'Eval{metric.name}': metric.result() for metric in eval_actor.metrics})

                  try:
                        video_file = f'{videos_dir}/video_{i}.mp4'
                        create_policy_eval_video(agent.policy, env, max_steps=120,
                                                 filename=video_file, max_envs_to_show=1)
                        iteration_logs['video'] = wandb.Video(video_file, fps=30, format="mp4")
                  except Exception as e:
                        print(f'Error creating video: {e}')

            collect_actor.run()
            print('Collect stats:')
            print(', '.join([f'{metric.name}: {metric.result():.3f}' for metric in collect_actor.metrics]))
            iteration_logs.update({metric.name: metric.result() for metric in collect_actor.metrics})

            loss_info = ppo_learner.run()
            print('Training info:')
            print(f'Loss: {loss_info.loss:.5f}, '
                  f'KL Penalty Loss: {loss_info.extra.kl_penalty_loss:.5f}, '
                  f'Entropy: {loss_info.extra.entropy_regularization_loss:.5f}, '
                  f'Value Estimation Loss: {loss_info.extra.value_estimation_loss:.5f}, '
                  f'PG Loss {loss_info.extra.policy_gradient_loss:.5f}')

            iteration_logs.update({
                  'loss': loss_info.loss.numpy(), 
                  **tf.nest.map_structure(lambda x: x.numpy(), loss_info.extra._asdict())
            })

            print()

            wandb.log(iteration_logs)
            replay_buffer.clear()

finally:
      wandb.finish()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Iteration: 0
Collect stats:
NumberOfEpisodes: 32.000, EnvironmentSteps: 2048.000, AverageReturn: 0.955, AverageEpisodeLength: 62.938
Training info:
Loss: 0.92742, KL Penalty Loss: 0.01046, Entropy: 0.00000, Value Estimation Loss: 0.95825, PG Loss -0.04129
Evaluation stats:
AverageReturn: 0.953, AverageEpisodeLength: 80.000



Iteration: 1
Collect stats:
NumberOfEpisodes: 30.000, EnvironmentSteps: 2048.000, AverageReturn: 0.946, AverageEpisodeLength: 66.700
Training info:
Loss: 0.47094, KL Penalty Loss: 0.00043, Entropy: 0.00000, Value Estimation Loss: 0.54559, PG Loss -0.07508

Iteration: 2
Collect stats:
NumberOfEpisodes: 30.000, EnvironmentSteps: 2048.000, AverageReturn: 0.953, AverageEpisodeLength: 65.600
Training info:
Loss: 0.10185, KL Penalty Loss: 0.00122, Entropy: 0.00000, Value Estimation Loss: 0.16324, PG Loss -0.06261

Iteration: 3
Collect stats:
NumberOfEpisodes: 34.000, EnvironmentSteps: 2048.000, AverageReturn: 0.956, AverageEpisodeLength: 59.765
Training info:
Loss: 0.11847, KL Penalty Loss: 0.00749, Entropy: 0.00000, Value Estimation Loss: 0.21133, PG Loss -0.10034

Iteration: 4
Collect stats:
NumberOfEpisodes: 31.000, EnvironmentSteps: 2049.000, AverageReturn: 0.954, AverageEpisodeLength: 63.581
Training info:
Loss: -0.00721, KL Penalty Loss: 0.00319, Entropy: 0.00000, Value Estimation Los

Evaluation stats:
AverageReturn: 0.953, AverageEpisodeLength: 80.000

Iteration: 6
Collect stats:
NumberOfEpisodes: 34.000, EnvironmentSteps: 2048.000, AverageReturn: 0.962, AverageEpisodeLength: 57.559
Training info:
Loss: 0.03863, KL Penalty Loss: 0.00297, Entropy: 0.00000, Value Estimation Loss: 0.14078, PG Loss -0.10512

Iteration: 7
Collect stats:
NumberOfEpisodes: 33.000, EnvironmentSteps: 2049.000, AverageReturn: 0.960, AverageEpisodeLength: 58.879
Training info:
Loss: 0.01563, KL Penalty Loss: 0.00520, Entropy: 0.00000, Value Estimation Loss: 0.14271, PG Loss -0.13228

Iteration: 8
Collect stats:
NumberOfEpisodes: 32.000, EnvironmentSteps: 2048.000, AverageReturn: 0.959, AverageEpisodeLength: 62.469
Training info:
Loss: 0.03292, KL Penalty Loss: 0.00018, Entropy: 0.00000, Value Estimation Loss: 0.15136, PG Loss -0.11862

Iteration: 9
Collect stats:
NumberOfEpisodes: 35.000, EnvironmentSteps: 2049.000, AverageReturn: 0.969, AverageEpisodeLength: 55.400
Training info:
Loss: -0.02


Iteration: 11
Collect stats:
NumberOfEpisodes: 37.000, EnvironmentSteps: 2048.000, AverageReturn: 0.959, AverageEpisodeLength: 54.541
Training info:
Loss: 0.02425, KL Penalty Loss: 0.00001, Entropy: 0.00000, Value Estimation Loss: 0.13519, PG Loss -0.11095

Iteration: 12
Collect stats:
NumberOfEpisodes: 42.000, EnvironmentSteps: 2049.000, AverageReturn: 0.975, AverageEpisodeLength: 47.095
Training info:
Loss: 0.03540, KL Penalty Loss: 0.00062, Entropy: 0.00000, Value Estimation Loss: 0.17071, PG Loss -0.13593

Iteration: 13
Collect stats:
NumberOfEpisodes: 43.000, EnvironmentSteps: 2049.000, AverageReturn: 0.972, AverageEpisodeLength: 46.605
Training info:
Loss: 0.19583, KL Penalty Loss: 0.00498, Entropy: 0.00000, Value Estimation Loss: 0.27726, PG Loss -0.08642

Iteration: 14
Collect stats:
NumberOfEpisodes: 43.000, EnvironmentSteps: 2049.000, AverageReturn: 0.972, AverageEpisodeLength: 45.558
Training info:
Loss: -0.01204, KL Penalty Loss: 0.00729, Entropy: 0.00000, Value Estimation

...

Evaluation stats:
AverageReturn: 0.996, AverageEpisodeLength: 37.000

Iteration: 21
Collect stats:
NumberOfEpisodes: 57.000, EnvironmentSteps: 2049.000, AverageReturn: 0.992, AverageEpisodeLength: 34.807
Training info:
Loss: -0.02823, KL Penalty Loss: 0.00000, Entropy: 0.00000, Value Estimation Loss: 0.09493, PG Loss -0.12315

Iteration: 22
Collect stats:
NumberOfEpisodes: 68.000, EnvironmentSteps: 2048.000, AverageReturn: 0.997, AverageEpisodeLength: 29.838
Training info:
Loss: -0.03970, KL Penalty Loss: 0.00193, Entropy: 0.00000, Value Estimation Loss: 0.10951, PG Loss -0.15115

Iteration: 23
Collect stats:
NumberOfEpisodes: 76.000, EnvironmentSteps: 2048.000, AverageReturn: 0.998, AverageEpisodeLength: 26.408
Training info:
Loss: -0.06981, KL Penalty Loss: 0.01519, Entropy: 0.00000, Value Estimation Loss: 0.08551, PG Loss -0.17051

Iteration: 24
Collect stats:
NumberOfEpisodes: 93.000, EnvironmentSteps: 2049.000, AverageReturn: 1.003, AverageEpisodeLength: 21.699
Training info:
Loss

Evaluation stats:
AverageReturn: 1.022, AverageEpisodeLength: 11.000

Iteration: 26
Waiting for debugger attach
Debugger attached


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


AverageEpisodeLength,▇██▇█▇▇▇▇▆▇▆▅▅▅▄▄▄▃▃▃▃▂▂▁▁
AverageReturn,▂▁▂▂▂▃▃▃▃▄▂▃▅▄▄▅▅▅▆▅▆▇▇▇██
EnvironmentSteps,▁▁▁▁██▁█▁█▁▁███▁██▁▁██▁▁██
EvalAverageEpisodeLength,███▆▄▁
EvalAverageReturn,▁▁▁▃▅█
NumberOfEpisodes,▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▃▃▃▄▄▄▄▅▆██
entropy_regularization_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
iteration,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
kl_penalty_loss,▆▁▂▄▂▂▂▃▁▂▁▁▁▃▄▁▁▁█▁▁▁▂█▂▁
l2_regularization_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▅▂▂▂▂▂▂▂▂▂▂▂▃▂▁▂▁▁▂▁▁▁▁▁▁


IndexError: list index out of range